In [1]:
import sys
sys.path.append('../..')
import numpy as np
from PIL import Image
import glob
import os
import shutil

# Pytorch
import torch
from torch import cuda

# DLBio and own scripts
import transforms_data as td
import ds_ear
import acquire_ear_dataset as a
from DLBio.pytorch_helpers import get_device


class Config():
    DATASET_DIR = '../dataset_low_res/'
    CATEGORIES = ds_ear.get_dataset(DATASET_DIR, transform_mode='size_only').classes
    # CATEGORIES = ["mila_wol", "falco_len", "jesse_kru", "konrad_von", "nils_loo", "johannes_boe", "johannes_wie", "sarah_feh", "janna_qua", "tim_moe"]
    DATA_TEST_DIR = "../auth_dataset/unknown-auth/*png"
    CATEGORIES.sort()
    AUTHORIZED = ["falco_len","konrad_von"]
    RESIZE_SMALL = True
    DEVICE = get_device()

model = torch.load('./class_sample/model.pt', Config.DEVICE)

['alissa_buh', 'falco_len', 'gregor_spi', 'janna_qua', 'jesse_kru', 'johannes_boe', 'johannes_wie', 'konrad_von', 'mila_wol', 'nils_loo', 'sarah_feh', 'tim_moe']


FileNotFoundError: [Errno 2] No such file or directory: './class_sample/model.pt'

In [ ]:
# Bilder aufnehmen
a.capture_ear_images(amount_pic=10, pic_per_stage=10, is_authentification=True)

In [ ]:
image_array = []
files = glob.glob(Config.DATA_TEST_DIR)
files.sort()
# declare function of transformation
preprocess = td.transforms_valid_and_test( td.get_resize(small=Config.RESIZE_SMALL) )

for f in files:
    image = Image.open(f)
    image_transformed = preprocess(image)
    image_transformed = image_transformed.reshape(
                            -1,
                            td.get_resize(small=Config.RESIZE_SMALL)[0],
                            td.get_resize(small=Config.RESIZE_SMALL)[1],
                            1
                            )
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    if cuda.is_available():
        image_array.append(image_transformed.type('torch.cuda.FloatTensor'))
    else:
        image_array.append(image_transformed.type('torch.FloatTensor'))

In [ ]:
all_classes = []
summ_pred = np.zeros(1)
for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()
		summ_pred = summ_pred + pred

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, Config.CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)
print(summ_pred)

In [ ]:

num_authorized = int(.3*len(image_array))
authentification_dict = {Config.CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:
    if a in Config.AUTHORIZED and summ_pred[0][Config.CATEGORIES.index(a)]>= num_authorized:
        print("Access granted! Welcome "  + a + "!")
        break
    else:
        print("Access denied")
        break


In [ ]:
shutil.rmtree('../auth_dataset/unknown-auth')